# Proceso de Limpieza de Datos de Arriendos en Puerto Montt

Este notebook toma los datos brutos extraídos por el script de web scraping (`arriendos_puerto_montt.xlsx`) y realiza un proceso completo de limpieza y transformación. El objetivo es preparar un dataset limpio y estructurado (`arriendos_puerto_montt_limpio.xlsx`) listo para ser analizado y visualizado en Power BI.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import pandas as pd
import numpy as np
import re
from API_UF import get_uf_from_mindicador
from unidecode import unidecode

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## 1. Carga de Datos

Cargamos el archivo Excel con los datos en crudo desde la carpeta `data`.

In [4]:
# --- Configuración de Archivos ---
input_path = r"../data/arriendos_puerto_montt.xlsx"
output_path = r"../data/arriendos_puerto_montt_limpio.xlsx"

# --- Cargar los Datos ---
try:
    df = pd.read_excel(input_path)
    print("Archivo Excel cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{input_path}'.")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo: {e}")

Archivo Excel cargado exitosamente.


## 2. Exploración Inicial

Visualizamos las primeras filas del DataFrame para entender la estructura y el formato de los datos antes de la limpieza.

In [5]:
# Mostramos las primeras 5 filas para entender la estructura inicial
df.head()

,Tipo_de_hogar,Titulo,Precio,Ubicacion,Atributos
0,Local en arriendo,Arriendo De Locales Comerciales Galeria Vinals...,UF 25,"Antonio Varas 986, Puerto Montt, Centro De Pue...",3 banos 625 m2 utiles
1,Departamento en arriendo,Arriendo Departamento Amoblado 3d1b1e En Valle...,$ 450.000,"Volcan Corcovado, Puerto Montt, La Paloma, Pue...",3 dormitorios 1 bano 42 m2 utiles
2,Casa en arriendo,"3d/2b/3e Amplia Casa En Valle Volcanes, Puerto...",$ 750.000,"Laguna Del Laja 4904, Puerto Montt, Puerto Montt",3 dormitorios 2 banos 100 m2 utiles
3,Departamento en arriendo,"2 Dormitorios, 1 Bano. 1 Estacionamiento. Buen...",$ 500.000,"Impecable Estado / Sargento Silva, Puerto Montt",2 dormitorios 1 bano 50 m2 utiles
4,Casa en arriendo,Casa En Parcela Esquina A Orilla De Carretera ...,$ 850.000,Casa En Parcela Esquina A Orilla De Carretera ...,3 dormitorios 2 banos 196 m2 utiles


## 3. Obtención del Valor de la UF

Para estandarizar los precios, necesitamos convertir los valores que están en UF a CLP. Para ello, llamamos a nuestro módulo `API_UF` que consulta el valor actualizado de la UF desde la API de `mindicador.cl`. Si la API falla, se utiliza un valor de respaldo.

In [6]:
# --- Obtener el Valor de la UF ---
print("Obteniendo valor de la UF...")
fecha, uf_actual = get_uf_from_mindicador()

if uf_actual:
    uf_actual = float(uf_actual)
    print(f"Valor de la UF para el {fecha.strftime('%d-%m-%Y')}: ${uf_actual:,.2f}")
else:
    # Si la API falla, usamos un valor de respaldo razonable
    uf_actual = 40000  
    print(f"Advertencia: No se pudo obtener la UF. Se usará un valor por defecto de ${uf_actual:,.0f}")

Obteniendo valor de la UF...
Valor de la UF para el 19-09-2025: $39,485.65


## 4. Limpieza y Conversión de Precios

Definimos una función para procesar la columna `Precio`. Esta función es capaz de:
- Identificar si un precio está en UF o CLP.
- Convertir los precios en UF a CLP multiplicándolos por el valor actual.
- Extraer únicamente los números de los precios en CLP, eliminando símbolos como `$` o puntos.

Luego, aplicamos esta función para crear una nueva columna `Precio_CLP` con todos los valores estandarizados.

In [7]:
def clean_price(price_str, uf_value):
    """
    Limpia una cadena de texto de precio, la convierte a CLP.
    Si el precio está en UF, lo convierte usando el valor de la UF.
    Si está en CLP, extrae solo los números.
    """
    price_str = str(price_str).strip()

    # Caso 1: Precio en UF
    if "uf" in price_str.lower():
        # Extraemos todos los números para manejar rangos (ej: "UF 20 - 22")
        numbers = re.findall(r"(\d+\.?\d*)", price_str)
        if numbers:
            try:
                numeric_values = [float(n) for n in numbers]
                uf_price_avg = sum(numeric_values) / len(numeric_values)
                return round(uf_price_avg * uf_value)
            except (ValueError, ZeroDivisionError):
                return np.nan
        return np.nan
    
    # Caso 2: Precio en CLP
    else:
        numbers_only = re.sub(r"\D", "", price_str)
        if numbers_only:
            try:
                return int(numbers_only)
            except ValueError:
                return np.nan
        return np.nan

# Aplicamos la función para crear la nueva columna 'Precio_CLP'
df['Precio_CLP'] = df['Precio'].apply(lambda x: clean_price(x, uf_actual))

# Verificamos el resultado
print("Columna 'Precio_CLP' creada.")
df[['Precio', 'Precio_CLP']].head(10)

Columna 'Precio_CLP' creada.


,Precio,Precio_CLP
0,UF 25,987141
1,$ 450.000,450000
2,$ 750.000,750000
3,$ 500.000,500000
4,$ 850.000,850000
5,$ 550.000,550000
6,$ 470.000,470000
7,$ 450.000,450000
8,"UF 18 , 52",1381998
9,$ 600.000,600000


## 5. Limpieza de Columnas de Texto

Normalizamos las columnas de texto principales (`Titulo`, `Ubicacion`, `Tipo_de_hogar`) para facilitar el análisis. Esto incluye:
- Eliminar tildes y caracteres especiales.
- Estandarizar el contenido, como quitar la frase "en arriendo" del tipo de hogar.

In [8]:
# Normalizamos texto: quitamos tildes y caracteres especiales
for col in ['Titulo', 'Ubicacion', 'Tipo_de_hogar']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x)))

# Estandarizamos la columna 'Tipo_de_hogar'
df['Tipo_de_hogar'] = df['Tipo_de_hogar'].str.replace(" en arriendo", "", case=False).str.strip()

print("Columnas de texto limpiadas.")
df[['Titulo', 'Tipo_de_hogar']].head()

Columnas de texto limpiadas.


,Titulo,Tipo_de_hogar
0,Arriendo De Locales Comerciales Galeria Vinals...,Local
1,Arriendo Departamento Amoblado 3d1b1e En Valle...,Departamento
2,"3d/2b/3e Amplia Casa En Valle Volcanes, Puerto...",Casa
3,"2 Dormitorios, 1 Bano. 1 Estacionamiento. Buen...",Departamento
4,Casa En Parcela Esquina A Orilla De Carretera ...,Casa


## 6. Extracción de Atributos

La columna `Atributos` contiene información valiosa (dormitorios, baños, superficie) en una sola cadena de texto. Usamos expresiones regulares para extraer cada uno de estos valores en columnas numéricas separadas, lo que permitirá realizar cálculos y filtros sobre ellos.

In [9]:
df['Atributos'] = df['Atributos'].astype(str)

# Usamos expresiones regulares para extraer la información de la columna 'Atributos'
df['Dormitorios'] = df['Atributos'].str.extract(r'(\d+)\s*dormitorio', flags=re.IGNORECASE).fillna(0)
df['Banos'] = df['Atributos'].str.extract(r'(\d+)\s*bano', flags=re.IGNORECASE).fillna(0)
df['Superficie_m2'] = df['Atributos'].str.extract(r'(\d+)\s*m2', flags=re.IGNORECASE).fillna(0)
df['Privados'] = df['Atributos'].str.extract(r'(\d+)\s*privado', flags=re.IGNORECASE).fillna(0)

# Convertimos las nuevas columnas a tipo numérico (entero)
df[['Dormitorios', 'Banos', 'Superficie_m2', 'Privados']] = df[['Dormitorios', 'Banos', 'Superficie_m2', 'Privados']].astype(int)

print("Atributos extraídos en nuevas columnas.")
df[['Atributos', 'Dormitorios', 'Banos', 'Superficie_m2']].head()

Atributos extraídos en nuevas columnas.


,Atributos,Dormitorios,Banos,Superficie_m2
0,3 banos 625 m2 utiles,0,3,625
1,3 dormitorios 1 bano 42 m2 utiles,3,1,42
2,3 dormitorios 2 banos 100 m2 utiles,3,2,100
3,2 dormitorios 1 bano 50 m2 utiles,2,1,50
4,3 dormitorios 2 banos 196 m2 utiles,3,2,196


## 7. Selección Final y Limpieza de Nulos

Creamos un nuevo DataFrame `df_limpio` que contiene únicamente las columnas que nos interesan para el análisis final. Además, eliminamos cualquier fila donde no se haya podido calcular un precio válido (`Precio_CLP`).

In [10]:
# Definimos las columnas que queremos en nuestro dataset final
columnas_finales = [
    'Tipo_de_hogar',
    'Precio',
    'Atributos',
    'Ubicacion',
    'Precio_CLP',
    'Dormitorios',
    'Banos',
    'Privados',
    'Superficie_m2'
]

# Creamos el DataFrame limpio con solo las columnas deseadas
df_limpio = df[columnas_finales].copy()

# Eliminamos filas donde no se pudo calcular el precio en CLP o es cero
df_limpio.dropna(subset=['Precio_CLP'], inplace=True)
df_limpio = df_limpio[df_limpio['Precio_CLP'] > 0]

print("DataFrame final creado y valores nulos eliminados.")

DataFrame final creado y valores nulos eliminados.


## 8. Verificación Final

Revisamos la información y los tipos de datos del DataFrame limpio para asegurarnos de que todo esté en el formato correcto antes de guardarlo.

In [11]:
# Usamos convert_dtypes() para que pandas asigne los tipos de datos más eficientes
df_limpio = df_limpio.convert_dtypes()

print("Tipos de datos finales:")
df_limpio.info()

Tipos de datos finales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tipo_de_hogar  597 non-null    string
 1   Precio         597 non-null    string
 2   Atributos      597 non-null    string
 3   Ubicacion      597 non-null    string
 4   Precio_CLP     597 non-null    Int64 
 5   Dormitorios    597 non-null    Int64 
 6   Banos          597 non-null    Int64 
 7   Privados       597 non-null    Int64 
 8   Superficie_m2  597 non-null    Int64 
dtypes: Int64(5), string(4)
memory usage: 45.0 KB


## 9. Guardado del Dataset Limpio

Finalmente, guardamos el DataFrame procesado en un nuevo archivo Excel en la carpeta `data`. Este archivo será el que se conectará a Power BI.

In [12]:
# --- Guardar el archivo limpio ---
try:
    df_limpio.to_excel(output_path, index=False)
    print(f"\n¡Éxito! Datos limpios guardados en:\n'{output_path}'")
except Exception as e:
    print(f"Ocurrió un error al guardar el archivo: {e}")


¡Éxito! Datos limpios guardados en:
'../data/arriendos_puerto_montt_limpio.xlsx'
